In [1]:
import pandas as pd
import json

In [2]:
from datasets import load_dataset

# Preparing the Datasets

The two datasets that were used here were: 
<p> note that for this dataset, I simply downloaded off huggingface hub since the load_dataset wasn't working immediately
<p>
https://huggingface.co/datasets/Hello-SimpleAI/HC3/blob/main/all.jsonl 


https://huggingface.co/datasets/MohamedRashad/ChatGPT-prompts/tree/main

This tutorial is going to be heavily influenced by this guide:
https://plainenglish.io/blog/creating-your-own-chatgpt-a-guide-to-fine-tuning-llms-with-lora

In [5]:
dataset = pd.read_json('datasets/Hello-SimpleAI-HC3-all.jsonl', lines=True)

In [6]:
dataset.head()

,question,human_answers,chatgpt_answers,index,source
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,reddit_eli5
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,reddit_eli5
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,reddit_eli5
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,reddit_eli5
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,reddit_eli5


In [7]:
len(dataset)

24322

In [8]:
# load the dataset
dataset2 = load_dataset('MohamedRashad/ChatGPT-prompts')

# save it to the directory that I want it at
dataset2['train'].to_json(path_or_buf='datasets/MohamedRashad-ChatGPT-prompts.jsonl',lines=True, orient="records")

Found cached dataset csv (/root/.cache/huggingface/datasets/MohamedRashad___csv/MohamedRashad--ChatGPT-prompts-29296547fda20fc1/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

438165

In [9]:
dataset2['train'][0]

{'human_prompt': 'Write a detailed product description for a food chopper tool that lets you chop fruits and vegetables.',
 'chatgpt_response': 'Introducing the ultimate kitchen tool for busy home cooks - the food chopper! This versatile gadget allows you to easily and efficiently chop a variety of fruits and vegetables, saving you time and energy in the kitchen.\nThe food chopper features a sharp, stainless steel blade that effortlessly slices through even the toughest produce, including carrots, onions, celery, and more. It also has a compact size that fits easily into most kitchen drawers, making it a convenient choice for even the smallest of kitchens.\nOne of the standout features of this food chopper is its adjustable slicing blade, which allows you to customize the thickness of your cuts. Whether you want thin, precise slices or thick, chunky pieces, the food chopper has you covered.\nThe food chopper also comes with a handy safety guard, which prevents the blade from accidental

In [10]:
len(dataset2['train'])

360

In [3]:
# Opening JSON file
f1 = open('datasets/Hello-SimpleAI-HC3-all.jsonl')
f2 = open('datasets/MohamedRashad-ChatGPT-prompts.jsonl')
Lines1 = f1.readlines()
Lines2 = f2.readlines()

questions, answers = [], []

In [4]:
# this dataset has columns for question, human_answers, and chatgpt_answers
for line in Lines1:
    row = json.loads(line)
    # pair up the questions and answers so that the human question goes in the questions list as "Human: "
    # and then the HUMAN answer goes in the answers list as "Assistant: "
    # we need to iterate through this in a for loop since there could be multiple answers in the list
    for answer in row["human_answers"]:
        questions.append("Human: "+row["question"])
        answers.append("Assistant: "+answer)
    # pair up the questions and answers so that the human question goes in the questions list as "Human: "
    # and then the CHATGPT answer goes in the answers list as "Assistant: "
    # we need to iterate through this in a for loop since there could be multiple answers in a list
    for answer in row["chatgpt_answers"]:
        questions.append("Human: "+row["question"])
        answers.append("Assistant: "+answer)

In [5]:
for line in Lines2:
    row = json.loads(line)
    questions.append("Human: " + row['human_prompt'])
    answers.append("Assistant: " + row['chatgpt_response'])

In [6]:
len(questions)

85809

In [7]:
df = pd.DataFrame()
df["question"] = questions
df["answer"] = answers

In [8]:
# shuffle the dataframe
df = df.sample(frac=1)

In [9]:
# Calculate the split index
split_idx = int(0.8 * len(df))

# Split the DataFrame
train_df = df[:split_idx]
val_df = df[split_idx:]

# Shuffle the datasets
train_df = train_df.sample(frac=1, random_state=42)
val_df = val_df.sample(frac=1, random_state=42)

# Save the dataframes
train_df.to_csv('datasets/train.csv', index=False)
val_df.to_csv('datasets/val.csv', index=False)

### To load the data

In [3]:
train_df = pd.read_csv('datasets/train.csv')
val_df = pd.read_csv('datasets/val.csv')

# Training in Lora

In [4]:
import os

import torch
from accelerate import Accelerator
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup, DataCollatorForSeq2Seq, BitsAndBytesConfig

from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model, get_peft_model_state_dict
from peft.utils.other import fsdp_auto_wrap_policy
from tqdm import tqdm


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart

In [5]:
accelerator = Accelerator()
# google/flan-t5-xxl
# google/flan-t5-small
model_name_or_path = "google/flan-t5-large"
batch_size = 2
max_length = 512
lr = 1e-4
num_epochs = 1
train_data = "./datasets/train.csv"
test_data = "./datasets/val.csv"

# implementing qlora research paper
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

peft_config = LoraConfig(
task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)
checkpoint_name = "chaT5_lora.pt"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = get_peft_model(model, peft_config)
accelerator.print(model.print_trainable_parameters())

dataset = load_dataset(
        'csv', data_files={
            "train": train_data,
            "validation": test_data,
        },
        cache_dir="./cache")


tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

trainable params: 2359296 || all params: 496102400 || trainable%: 0.47556633469219256
None


Found cached dataset csv (/workspace/notebooks/cache/csv/default-dc4ba4db0dc8058b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["question"]]
    model_inputs = tokenizer(
        inputs, max_length=max_length, padding=True, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["answer"], max_length=max_length, padding=True, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
with accelerator.main_process_first():
    processed_datasets = dataset.map(
        preprocess_function,
        batched=True,
        num_proc=16,
        remove_columns=dataset["train"].column_names,
        load_from_cache_file=False,
        desc="Running tokenizer on dataset",
    )

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

Running tokenizer on dataset (num_proc=16):   0%|          | 0/68647 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3614: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3614: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3614: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Running tokenizer on dataset (num_proc=16):   0%|          | 0/17162 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3614: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3614: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3614: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [8]:
data_collator = DataCollatorForSeq2Seq(
        tokenizer, model=model)

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=data_collator, batch_size=batch_size, pin_memory=True
)

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [9]:
len(train_dataloader)

34324

In [10]:
if getattr(accelerator.state, "fsdp_plugin", None) is not None:
    accelerator.state.fsdp_plugin.auto_wrap_policy = fsdp_auto_wrap_policy(model)

model, train_dataloader, eval_dataloader, optimizer, lr_scheduler = accelerator.prepare(
    model, train_dataloader, eval_dataloader, optimizer, lr_scheduler
)
accelerator.print(model)
#accelerator.state.deepspeed_plugin.zero_stage == 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        if step%1000 == 0:
            print("loss: ",loss.detach().float())
            accelerator.wait_for_everyone()
            if accelerator.is_main_process:
                accelerator.save(
                    get_peft_model_state_dict(model, state_dict=accelerator.get_state_dict(model)), checkpoint_name
                )


    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        preds = accelerator.gather_for_metrics(torch.argmax(outputs.logits, -1)).detach().cpu().numpy()
        eval_preds.extend(tokenizer.batch_decode(preds, skip_special_tokens=True))
    eval_epoch_loss = eval_loss / len(train_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(eval_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    accelerator.print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

    accelerator.wait_for_everyone()
    accelerator.save(
        get_peft_model_state_dict(model, state_dict=accelerator.get_state_dict(model)), checkpoint_name
    )
    accelerator.wait_for_everyone()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear4bit(
                    in_features=1024, out_features=1024, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedd

  0%|          | 1/34324 [00:00<7:39:09,  1.25it/s]

loss:  tensor(27.5000, device='cuda:0')


  3%|▎         | 1001/34324 [03:17<1:55:20,  4.81it/s]

loss:  tensor(nan, device='cuda:0')


...

loss:  tensor(nan, device='cuda:0')


 99%|█████████▉| 34001/34324 [1:51:31<01:04,  5.04it/s]

loss:  tensor(nan, device='cuda:0')


100%|██████████| 8581/8581 [11:30<00:00, 12.43it/s]

epoch=0: train_ppl=tensor(nan, device='cuda:0') train_epoch_loss=tensor(nan, device='cuda:0') eval_ppl=tensor(nan, device='cuda:0') eval_epoch_loss=tensor(nan, device='cuda:0')


In [11]:
# how I figured this out:
# https://huggingface.co/blog/peft
model.save_pretrained("flan-t5-large-lora") 

In [ ]:
# then put it in the loras folder of the oobabooga webui and then run this:
# python server.py --model google_flan-t5-small --lora luke_test